# 201: Exampville Mode Choice

Welcome to Exampville, the best simulated town in this here part of the internet!

Exampville is a demonstration provided with Larch that walks through some of the 
data and tools that a transportation planner might use when building a travel model. 

In [ ]:
import larch, numpy, pandas, os
from larch import P, X

In [ ]:
larch.__version__

In this example notebook, we will walk through the creation of a tour mode choice model.

To begin, we'll re-load the tours and skims data from the 
data setup example.

In [ ]:
hh, pp, tour, skims = larch.example(200, ['hh', 'pp', 'tour', 'skims'])

## Preprocessing

The Exampville data output contains a set of files similar to what we might
find for a real travel survey: network skims, and tables of households, persons,
and tours.  We'll need to merge these tables to create a composite dataset
for mode choice model estimation.

We can merge data from other tables using the usual pandas syntax for merging.


In [ ]:
raw = tour.merge(hh, on='HHID').merge(pp, on=('HHID', 'PERSONID'))

Our zone numbering system starts with zone 1, as is common for many TAZ numbering 
systems seen in practice.  But, for looking up data in the skims matrix, we'll need
to use zero-based numbering that is standard in Python.  So we'll create two new 
TAZ-index columns to assist this process.

In [ ]:
raw["HOMETAZi"] = raw["HOMETAZ"] - 1
raw["DTAZi"] = raw["DTAZ"] - 1

In [ ]:
raw.info()

In Exampville, there are only two kinds of trips: 

- work (purpose=1) and 
- non-work (purpose=2). 

We want to estimate a mode choice model for work trips, 
so we’ll begin by excluding all the other trips:

In [ ]:
raw = raw[raw.TOURPURP == 1]

In [ ]:
raw.info()

In [ ]:
skims

For this tour mode choice model, we can pick values 
out of the skims for the known O-D of the tour, so 
that we have access to the level-of-service data for
each possible mode serving that O-D pair.  We'll 
use the `get_rc_dataframe` method of the `larch.OMX`
object, which lets us give the a list of the index for the 
production and attraction (row and column) value we
want to use.  

In [ ]:
raw = raw.join(
    skims.get_rc_dataframe(
        raw.HOMETAZi, raw.DTAZi,
    )
)

We can review the `raw` frame to see what variables are now included.

In [ ]:
raw.info()

Now we can assemble this data, along with the definitions of the alternatives,
into a `larch.DataFrames` object.  This object links information about the 
raw data and the choices into one data structure.  It can handle multiple data
formats (including `idco` and `idca` data simultanously), but here we'll only
be using a single `idco` format data table.

In [ ]:
# For clarity, we can define numbers as names for modes
DA = 1
SR = 2
Walk = 3
Bike = 4
Transit = 5

In [ ]:
dfs = larch.DataFrames(
    co=raw, 
    alt_codes=[DA,SR,Walk,Bike,Transit], 
    alt_names=['DA','SR','Walk','Bike','Transit'],
    ch_name='TOURMODE',
)

## Model Definition

And then we are ready to create our model.

In [ ]:
m = larch.Model(dataservice=dfs)
m.title = "Exampville Work Tour Mode Choice v1"

We will explicitly define the set of utility functions 
we want to use.  Because the DataFrames we are using to 
serve data to this model contains exclusively `idco` format
data, we'll use only the `utility_co` mapping to define
a unique utility function for each alternative.

In [ ]:
m.utility_co[DA] = (
        + P.InVehTime * X.AUTO_TIME
        + P.Cost * X.AUTO_COST # dollars per mile
)

m.utility_co[SR] = (
        + P.ASC_SR
        + P.InVehTime * X.AUTO_TIME
        + P.Cost * (X.AUTO_COST * 0.5) # dollars per mile, half share
        + P("LogIncome:SR") * X("log(INCOME)")
)

m.utility_co[Walk] = (
        + P.ASC_Walk
        + P.NonMotorTime * X.WALK_TIME
        + P("LogIncome:Walk") * X("log(INCOME)")
)

m.utility_co[Bike] = (
        + P.ASC_Bike
        + P.NonMotorTime * X.BIKE_TIME
        + P("LogIncome:Bike") * X("log(INCOME)")
)

m.utility_co[Transit] = (
        + P.ASC_Transit
        + P.InVehTime * X.TRANSIT_IVTT
        + P.OutVehTime * X.TRANSIT_OVTT
        + P.Cost * X.TRANSIT_FARE
        + P("LogIncome:Transit") * X('log(INCOME)')
)

To write a nested logit mode, we'll attach some nesting nodes to the 
model's `graph`.  Each `new_node` allows us to define the set of 
codes for the child nodes (elemental alternatives, or lower level nests)
as well as giving the new nest a name and assigning a logsum parameter.
The return value of this method is the node code for the newly created 
nest, which then can potenially be used as a child code when creating
a higher level nest.  We do this below, adding the 'Car' nest into the 
'Motor' nest.

In [ ]:
Car = m.graph.new_node(parameter='Mu:Car', children=[DA,SR], name='Car')
NonMotor = m.graph.new_node(parameter='Mu:NonMotor', children=[Walk,Bike], name='NonMotor')
Motor = m.graph.new_node(parameter='Mu:Motor', children=[Car,Transit], name='Motor')

Let's visually check on the nesting structure.

In [ ]:
m.graph

The tour mode choice model's choice variable is indicated by 
the code value in 'TOURMODE', and this can be 
defined for the model using `choice_co_code`.

In [ ]:
m.choice_co_code = 'TOURMODE'

We can also give a dictionary of availability conditions based 
on values in the `idco` data, using the `availability_co_vars`
attribute.  Alternatives that are always available can be indicated
by setting the criterion to 1.

In [ ]:
m.availability_co_vars = {
    DA: 'AGE >= 16',
    SR: 1,
    Walk: 'WALK_TIME < 60',
    Bike: 'BIKE_TIME < 60',
    Transit: 'TRANSIT_FARE>0',
}

Then let's prepare this data for estimation.  Even though the
data is already in memory, the `load_data` method is used to 
pre-process the data, extracting the required values, pre-computing 
the values of fixed expressions, and assembling the results into
contiguous arrays suitable for computing the log likelihood values
efficiently.

## Model Estimation

In [ ]:
m.load_data()

We can check on some important statistics of this loaded data even
before we estimate the model.

In [ ]:
m.dataframes.choice_avail_summary()

In [ ]:
m.dataframes.data_co.statistics()

If we are satisfied with the statistics we see above, we
can go ahead and estimate the model.

In [ ]:
result = m.maximize_loglike(method='slsqp')

After we find the best fitting parameters, we can compute
some variance-covariance statistics, incuding standard errors of
the estimates and t statistics, using `calculate_parameter_covariance`.

In [ ]:
m.calculate_parameter_covariance()

Then we can review the results in a variety of report tables.

In [ ]:
m.parameter_summary()

In [ ]:
m.estimation_statistics()

## Save and Report Model

In [ ]:
report = larch.Reporter(title=m.title)

In [ ]:
report << '# Parameter Summary' << m.parameter_summary()

In [ ]:
report << "# Estimation Statistics" << m.estimation_statistics()

In [ ]:
report << "# Utility Functions" << m.utility_functions()

In [ ]:
report.save(
    '/tmp/exampville_mode_choice.html', 
    overwrite=True, 
    metadata=m,
)